## Learning Lyapunov function for Cart Inverted Pendulum System

In [1]:
# -*- coding: utf-8 -*-
from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt

## Neural network model
Building NN with random parameters for Lyapunov function and initializing parameters of NN controller to LQR solution

LQR solution is obtained by minimizing the cost function J = ∫(xᵀQx + uᵀRu)dt, where Q is 4×4 identity matrix and R is 1×1 identity matrix

In [2]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output,lqr):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.control = torch.nn.Linear(n_input,1,bias=False)
        self.control.weight = torch.nn.Parameter(lqr)

    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        u = self.control(x)
        return out,u

## Dynamical system

In [3]:
def f_value(x,u):
    #Dynamics
    y = []
    g = 9.81  # gravity
    l = 0.7   # length of the pole 
    M = 1   # cart mass
    m = 0.5  # Pendulum mass
    
    for r in range(0,len(x)): 
        f = [ x[r][1], ((0.5*g*m*np.sin(2*x[r][2]))-(l*m*((x[r][3])**2)*np.sin(x[r][2])))/((m*(np.sin(x[r][2])**2))+M), 
             x[r][3], ((g*(M+m)*np.sin(x[r][2]))-(l*m*((x[r][3])**2)*np.sin(x[r][2])*np.cos(x[r][2])))/(l*((m*(np.sin(x[r][2])**2))+M))]
        y.append(f)                                                                                   
    y = torch.tensor(y)
    y[:,1] = y[:,1] + ((u[1])/((m*((np.sin(x[r][2]))**2))+M))
    y[:,3] = y[:,3] + ((u[3]*np.cos(x[r][2]))/(l*((m*((np.sin(x[r][2]))**2))+M)))
    return y

## Options

In [4]:
'''
For learning 
'''
N = 200             # sample size
D_in = 4            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension
torch.manual_seed(10)  
x = torch.Tensor(N, D_in).uniform_(-4, 4)           
x_0 = torch.zeros([1, 4])

'''
For verifying 
'''
x1 = Variable("x1")
x2 = Variable("x2")
x3 = Variable("x3")
x4 = Variable("x4")
vars_ = [x1,x2,x3,x4]
G = 9.81 
l = 0.7  
M = 1
m = 0.5
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
epsilon = 0
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = -3
ball_ub = 3

In [5]:
x

tensor([[-0.3353, -0.1371, -1.5000,  0.9202],
        [-2.2884, -0.7054,  1.5504,  3.7545],
        [ 0.9424, -1.3571,  0.3835, -0.4483],
        [ 1.6327,  0.4580,  1.5669,  3.8794],
        [-1.6610, -0.1414,  0.9199, -0.0261],
        [-0.3831, -3.5399, -3.4504, -3.5994],
        [-3.9139, -3.7254, -3.0302, -3.6077],
        [-3.7523,  1.7535,  2.4534,  2.7029],
        [ 2.1549,  1.3554,  1.7622, -2.2121],
        [ 3.6014, -0.2759,  3.4512,  1.2263],
        [ 3.1309,  3.1908, -0.8357, -1.1634],
        [ 0.6019, -0.1700,  0.6257,  2.0292],
        [-3.1257, -0.1836, -3.1393,  3.8629],
        [-2.8132,  0.7650, -1.0929,  2.2736],
        [ 0.0134, -0.4023,  2.9278,  3.6535],
        [-2.9029, -3.8581,  0.3334,  1.2600],
        [ 0.9129,  3.6949,  1.7955, -1.8401],
        [-2.7393,  3.1033,  3.8337, -1.8981],
        [-2.5600,  1.4001, -2.8607, -0.9679],
        [-3.9563,  1.0943, -1.3639, -2.2379],
        [-2.5436, -1.4075,  1.8998,  0.2265],
        [ 3.4446, -1.4297, -1.1706

# Learning and Falsification

In [6]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    lqr = torch.tensor([[-3.16227766016952, -129.316379970420, 10041.1520750571, 227.861329326708]])    # lqr solution
    model = Net(D_in,H1, D_out,lqr)
    L = []
    i = 0 
    t = 0
    max_iters = 2000
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid: 
        V_candidate, u = model(x)
        X0,u0 = model(x_0)
        f = f_value(x,u)
        Circle_Tuning = Tune(x)
        # Compute lie derivative of V : L_V =| ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)

        # With tuning term 
        Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()\
                    +2.2*((Circle_Tuning-6*V_candidate).pow(2)).mean()+(X0).pow(2) 
        # Without tuning term
        #Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)
        
        
        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        w1 = model.layer1.weight.data.numpy()
        w2 = model.layer2.weight.data.numpy()
        b1 = model.layer1.bias.data.numpy()
        b2 = model.layer2.bias.data.numpy()
        q = model.control.weight.data.numpy()

        # Falsification
        if i % 10 == 0:
            u_NN = (q.item(0)*x1 + q.item(1)*x2 + q.item(2)*x3 + q.item(3)*x4) 
            f = [x2,(((0.5*G*m*sin(2*(x3)))-(l*m*((x4)**2)*sin(x3))+ u_NN )/((m*(sin(x3)**2))+M)),
                 x4,(G*(M+m)*sin(x3)-l*m*((x4)**2)*sin(x3)*cos(x3) + u_NN)/(l*(m*(sin(x3)**2)+M))]
            
            #Candidate V
            z1 = np.dot(vars_,w1.T)+b1

            a1 = []
            for j in range(0,len(z1)):
                a1.append(tanh(z1[j]))
            z2 = np.dot(a1,w2.T)+b2
            V_learn = tanh(z2.item(0))

            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            result= CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config,epsilon)
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = AddCounterexamples(x,result,10)
            else:  
                valid = True
                print("Satisfy conditions!!")
                print(V_learn, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 
        i += 1

    stop = timeit.default_timer()


    np.savetxt("w1.txt", model.layer1.weight.data, fmt="%s")
    np.savetxt("w2.txt", model.layer2.weight.data, fmt="%s")
    np.savetxt("b1.txt", model.layer1.bias.data, fmt="%s")
    np.savetxt("b2.txt", model.layer2.bias.data, fmt="%s")
    np.savetxt("q.txt", model.control.weight.data, fmt="%s")

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

0 Lyapunov Risk= 256.9264221191406
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.724650638949731807, -1.723651169911057712]
x2 : [1.535149560570235883, 1.535700069367415654]
x3 : [-0.00887723797470856002, -0.007890878199740941276]
x4 : [1.914974324935228678, 1.915873987519819854]
1 Lyapunov Risk= 3688.731689453125
2 Lyapunov Risk= 3230.07421875
3 Lyapunov Risk= 2690.071044921875
4 Lyapunov Risk= 2123.233642578125
5 Lyapunov Risk= 1562.1505126953125
6 Lyapunov Risk= 1052.18115234375
7 Lyapunov Risk= 639.3193969726562
8 Lyapunov Risk= 359.1178283691406
9 Lyapunov Risk= 201.85711669921875
10 Lyapunov Risk= 117.81201934814453
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
11 Lyapunov Risk= 53.58365249633789
12 Lyapun

121 Lyapunov Risk= 6.830289363861084
122 Lyapunov Risk= 6.786568641662598
123 Lyapunov Risk= 6.743598937988281
124 Lyapunov Risk= 6.701437473297119
125 Lyapunov Risk= 6.659992694854736
126 Lyapunov Risk= 6.619019508361816
127 Lyapunov Risk= 6.578906059265137
128 Lyapunov Risk= 6.539431095123291
129 Lyapunov Risk= 6.500582218170166
130 Lyapunov Risk= 6.4623308181762695
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
131 Lyapunov Risk= 6.255361080169678
132 Lyapunov Risk= 6.219205856323242
133 Lyapunov Risk= 6.183577060699463
134 Lyapunov Risk= 6.148609161376953
135 Lyapunov Risk= 6.11423397064209
136 Lyapunov Risk= 6.080419063568115
137 Lyapunov Risk= 6.047145843505859
138 Lyapunov Risk= 6.014449119567871
139 Lyapunov Risk= 5.982241153717041
140 Lyapunov R

242 Lyapunov Risk= 3.433913230895996
243 Lyapunov Risk= 3.4286961555480957
244 Lyapunov Risk= 3.423532485961914
245 Lyapunov Risk= 3.418423652648926
246 Lyapunov Risk= 3.4133591651916504
247 Lyapunov Risk= 3.4083445072174072
248 Lyapunov Risk= 3.403379440307617
249 Lyapunov Risk= 3.39846134185791
250 Lyapunov Risk= 3.3935887813568115
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
251 Lyapunov Risk= 3.3292508125305176
252 Lyapunov Risk= 3.3246009349823
253 Lyapunov Risk= 3.3199992179870605
254 Lyapunov Risk= 3.3154449462890625
255 Lyapunov Risk= 3.3109323978424072
256 Lyapunov Risk= 3.30646014213562
257 Lyapunov Risk= 3.3021676540374756
258 Lyapunov Risk= 3.297663450241089
259 Lyapunov Risk= 3.2933287620544434
260 Lyapunov Risk= 3.289030075073242
========

362 Lyapunov Risk= 2.536024570465088
363 Lyapunov Risk= 2.5343282222747803
364 Lyapunov Risk= 2.5326435565948486
365 Lyapunov Risk= 2.5309698581695557
366 Lyapunov Risk= 2.5293073654174805
367 Lyapunov Risk= 2.5276546478271484
368 Lyapunov Risk= 2.5260095596313477
369 Lyapunov Risk= 2.5243749618530273
370 Lyapunov Risk= 2.522749900817871
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
371 Lyapunov Risk= 2.4887049198150635
372 Lyapunov Risk= 2.487121105194092
373 Lyapunov Risk= 2.4855480194091797
374 Lyapunov Risk= 2.48398494720459
375 Lyapunov Risk= 2.4824306964874268
376 Lyapunov Risk= 2.4808855056762695
377 Lyapunov Risk= 2.47935152053833
378 Lyapunov Risk= 2.4778265953063965
379 Lyapunov Risk= 2.4763102531433105
380 Lyapunov Risk= 2.4748024940490723
==

481 Lyapunov Risk= 2.0793569087982178
482 Lyapunov Risk= 2.0785775184631348
483 Lyapunov Risk= 2.0777997970581055
484 Lyapunov Risk= 2.0770254135131836
485 Lyapunov Risk= 2.0762550830841064
486 Lyapunov Risk= 2.0754873752593994
487 Lyapunov Risk= 2.0747218132019043
488 Lyapunov Risk= 2.073960781097412
489 Lyapunov Risk= 2.07320237159729
490 Lyapunov Risk= 2.072446346282959
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
491 Lyapunov Risk= 2.050811290740967
492 Lyapunov Risk= 2.050069808959961
493 Lyapunov Risk= 2.0493311882019043
494 Lyapunov Risk= 2.0486032962799072
495 Lyapunov Risk= 2.047877311706543
496 Lyapunov Risk= 2.047154664993286
497 Lyapunov Risk= 2.046434164047241
498 Lyapunov Risk= 2.0457170009613037
499 Lyapunov Risk= 2.045001745223999
500 L

...

1481 Lyapunov Risk= 1.2206131219863892
1482 Lyapunov Risk= 1.2190368175506592
1483 Lyapunov Risk= 1.2183090448379517
1484 Lyapunov Risk= 1.2175827026367188
1485 Lyapunov Risk= 1.2168582677841187
1486 Lyapunov Risk= 1.2161353826522827
1487 Lyapunov Risk= 1.2154135704040527
1488 Lyapunov Risk= 1.2146937847137451
1489 Lyapunov Risk= 1.2139703035354614
1490 Lyapunov Risk= 1.2132484912872314
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
1491 Lyapunov Risk= 1.2099308967590332
1492 Lyapunov Risk= 1.210685133934021
1493 Lyapunov Risk= 1.2085003852844238
1494 Lyapunov Risk= 1.2077866792678833
1495 Lyapunov Risk= 1.2070742845535278
1496 Lyapunov Risk= 1.2063566446304321
1497 Lyapunov Risk= 1.2056387662887573
1498 Lyapunov Risk= 1.2049230337142944
1499 Lyapunov Ri

1601 Lyapunov Risk= 1.1094334125518799
1602 Lyapunov Risk= 1.108797311782837
1603 Lyapunov Risk= 1.1081631183624268
1604 Lyapunov Risk= 1.1075303554534912
1605 Lyapunov Risk= 1.1086907386779785
1606 Lyapunov Risk= 1.1063761711120605
1607 Lyapunov Risk= 1.1058114767074585
1608 Lyapunov Risk= 1.1052457094192505
1609 Lyapunov Risk= 1.1046764850616455
1610 Lyapunov Risk= 1.1041045188903809
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
1611 Lyapunov Risk= 1.101574420928955
1612 Lyapunov Risk= 1.100999116897583
1613 Lyapunov Risk= 1.1004228591918945
1614 Lyapunov Risk= 1.0998446941375732
1615 Lyapunov Risk= 1.0992658138275146
1616 Lyapunov Risk= 1.0986864566802979
1617 Lyapunov Risk= 1.0981063842773438
1618 Lyapunov Risk= 1.0975425243377686
1619 Lyapunov Risk

1721 Lyapunov Risk= 1.0100663900375366
1722 Lyapunov Risk= 1.025193691253662
1723 Lyapunov Risk= 1.0247677564620972
1724 Lyapunov Risk= 1.0258158445358276
1725 Lyapunov Risk= 1.0238860845565796
1726 Lyapunov Risk= 1.0234707593917847
1727 Lyapunov Risk= 1.0229862928390503
1728 Lyapunov Risk= 1.022526502609253
1729 Lyapunov Risk= 1.022051215171814
1730 Lyapunov Risk= 1.0215624570846558
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
1731 Lyapunov Risk= 1.0215083360671997
1732 Lyapunov Risk= 1.0186069011688232
1733 Lyapunov Risk= 1.0180879831314087
1734 Lyapunov Risk= 1.0176461935043335
1735 Lyapunov Risk= 1.0171887874603271
1736 Lyapunov Risk= 1.016742467880249
1737 Lyapunov Risk= 1.0163137912750244
1738 Lyapunov Risk= 1.0158770084381104
1739 Lyapunov Risk=

1841 Lyapunov Risk= 0.9525834918022156
1842 Lyapunov Risk= 0.9521974325180054
1843 Lyapunov Risk= 0.9517951607704163
1844 Lyapunov Risk= 0.9539848566055298
1845 Lyapunov Risk= 0.9527000188827515
1846 Lyapunov Risk= 0.9505400657653809
1847 Lyapunov Risk= 0.9511306285858154
1848 Lyapunov Risk= 0.9497374296188354
1849 Lyapunov Risk= 0.9493270516395569
1850 Lyapunov Risk= 0.9449913501739502
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
1851 Lyapunov Risk= 0.9484953880310059
1852 Lyapunov Risk= 0.9471228122711182
1853 Lyapunov Risk= 0.9511101245880127
1854 Lyapunov Risk= 0.9464000463485718
1855 Lyapunov Risk= 0.9494454860687256
1856 Lyapunov Risk= 0.9522565007209778
1857 Lyapunov Risk= 0.9331624507904053
1858 Lyapunov Risk= 0.9440633058547974
1859 Lyapunov R

1961 Lyapunov Risk= 0.8873072266578674
1962 Lyapunov Risk= 0.8869338631629944
1963 Lyapunov Risk= 0.8865540027618408
1964 Lyapunov Risk= 0.8887389302253723
1965 Lyapunov Risk= 0.8887577056884766
1966 Lyapunov Risk= 0.8854531049728394
1967 Lyapunov Risk= 0.8851718306541443
1968 Lyapunov Risk= 0.885784387588501
1969 Lyapunov Risk= 0.8845644593238831
1970 Lyapunov Risk= 0.8872637152671814
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-1.840305303174620821, -1.839585940103181549]
x2 : [1.568713922387883652, 1.569551830486991539]
x3 : [-1.322374658220856869, -1.321379474986198455]
x4 : [1.184787361118105231, 1.185342523845692941]
1971 Lyapunov Risk= 0.8823926448822021
1972 Lyapunov Risk= 0.8820350170135498
1973 Lyapunov Risk= 0.8816642761230469
1974 Lyapunov Risk= 0.8805664777755737
1975 Lyapunov Risk= 0.8700019121170044
1976 Lyapunov Risk= 0.8804848194122314
1977 Lyapunov Risk= 0.8821791410446167
1978 Lyapunov Risk= 0.8772693276405334
1979 Lyapunov Ri

#### Checking result with smaller epsilon ( Lie derivative of V <= epsilon )

In [7]:
epsilon = -0.00001
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config, epsilon)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
else:  
    print("Satisfy conditions with epsilon= ",epsilon)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)

TypeError: unsupported operand type(s) for *: 'Tensor' and 'dreal._dreal_py.Expression'

### More details on Lyapunov risk
Generally, we start training with Lyapunov risk without the tuning term.      
For example, (1* F.relu(-V_candidate)+ 1.5* F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)    
The weight of each term (1, 1.5, 1.2) can be tuned for balancing each Lyapunov condition.     
Furthermore, using F.relu(L_V+0.5) allows the learning procedure to seek a candidate Lyapunov function with more negative Lie derivative.   
Here 0.5 is also a tunable parameter based on your goal.    
In this example, we use Lyapunov risk with tuning term for achieving large ROA     

## 